### Code for Training timeleft Neural Network

In [166]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from sklearn import preprocessing as preproc

import os
import random

In [172]:
NAME = "TIME_PASSED_NN_1"
dir_name = os.path.join("logs", NAME)
tensorboard = Tensorboard = TensorBoard(log_dir=dir_name)

In [155]:
#Load Data
DATA_DIR = "DataSets"
file_name = "Time Remaining In Activity Data Set_3.xlsx"
file_name = os.path.join(DATA_DIR, file_name)

df = pd.read_excel(file_name, sheet_name='Sheet3')
activity_dic = {1: "Beast Mode",
               2: "Studying",
               3: "Beach",
               4: "Relaxing"}
df.head()

,Activity 1,Activity 2,Time Remaining Percentage,Energy Change
0,1,1,0.0041,1
1,1,1,0.0082,1
2,1,1,0.0123,1
3,1,1,0.0164,1
4,1,1,0.0205,1


In [156]:
# Preprocess Data
def preproc_df(df):
    ### ONE HOT ENCODING FOR ACTIVITYS
    act1_cols = ["act11", "act12", "act13", "act14"]
    act2_cols = ["act21", "act22", "act23", "act24"]
    
    actcols = act1_cols+act2_cols
    df_r, df_c = df.shape
    
    activity_dic = {}
    
    for col in actcols:
        activity_dic[col] = [0]*df_r
    
    act_df = pd.DataFrame(activity_dic, columns = actcols)
    
    for index, row in act_df.iterrows():
        act1_in = int(df["Activity 1"].iloc[index])
        act2_in = int(df["Activity 2"].iloc[index])        
        row["act1" + str(act1_in)] = 1 
        row["act2" + str(act2_in)] = 1 
        
    # Concatnate one hot encoding df to regular df
    
    df = pd.concat([df,act_df], axis=1)
    df.dropna(inplace=True)
    
    data = []
    for row in df.values:
        EnergyChange = row[3]
        activitys = row[-8:]
        timeleft = row[2]
        xdata = list(activitys)+[timeleft]
        data.append([np.array(xdata),EnergyChange])
        
    random.shuffle(data)
        
    X_data = []
    y_data = []
    
    for X,y in data:
        X_data.append(X)
        y_data.append(y)
    
        
    return np.array(X_data), np.array(y_data),df

In [157]:
X,y,ndf= preproc_df(df)
ndf.head()
# X[0]

,Activity 1,Activity 2,Time Remaining Percentage,Energy Change,act11,act12,act13,act14,act21,act22,act23,act24
0,1,1,0.0041,1,1,0,0,0,1,0,0,0
1,1,1,0.0082,1,1,0,0,0,1,0,0,0
2,1,1,0.0123,1,1,0,0,0,1,0,0,0
3,1,1,0.0164,1,1,0,0,0,1,0,0,0
4,1,1,0.0205,1,1,0,0,0,1,0,0,0


In [203]:
# Making Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(24, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(24, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(24, activation=tf.nn.relu))
# model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [204]:
model.fit(X,y,epochs=10,validation_split=0.2,callbacks = [tensorboard])

Train on 3456 samples, validate on 384 samples
Epoch 1/10
3456/3456 [==============================] - 1s 163us/sample - loss: 0.9944 - accuracy: 0.5347 - val_loss: 0.8983 - val_accuracy: 0.6042
Epoch 2/10
3456/3456 [==============================] - 0s 48us/sample - loss: 0.8254 - accuracy: 0.6218 - val_loss: 0.7635 - val_accuracy: 0.6380
Epoch 3/10
3456/3456 [==============================] - 0s 59us/sample - loss: 0.6686 - accuracy: 0.7031 - val_loss: 0.6007 - val_accuracy: 0.7422
Epoch 4/10
3456/3456 [==============================] - 0s 55us/sample - loss: 0.5042 - accuracy: 0.7995 - val_loss: 0.4582 - val_accuracy: 0.8490
Epoch 5/10
3456/3456 [==============================] - 0s 55us/sample - loss: 0.3798 - accuracy: 0.8782 - val_loss: 0.3692 - val_accuracy: 0.9193
Epoch 6/10
3456/3456 [==============================] - 0s 58us/sample - loss: 0.3007 - accuracy: 0.9123 - val_loss: 0.3065 - val_accuracy: 0.9323
Epoch 7/10
3456/3456 [==============================] - 0s 59us/sample

In [190]:
## Save Model
MODEL_DIR = "models"
model_name = "NN_Energy_change_picker_less_layers"
file_name = os.path.join(MODEL_DIR, model_name)
model.save(file_name)

INFO:tensorflow:Assets written to: models\NN_Energy_change_picker_less_layers\assets
